In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.optimize import root
from IPython.display import display, Markdown, Image

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 12, 'figure.figsize': (10, 6), 'figure.dpi': 130,
                     'axes.titlesize': 'x-large', 'axes.labelsize': 'large',
                     'xtick.labelsize': 'medium', 'ytick.labelsize': 'medium'})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-block alert-info'>📝 **Note:** {msg}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized for General Equilibrium analysis.")

# Part 5: Microeconomic Theory
## Chapter 5.02: General Equilibrium: From Theory to Computation

### Table of Contents
1.  [The Arrow-Debreu Model and Existence of Equilibrium](#1.-The-Arrow-Debreu-Model-and-Existence-of-Equilibrium)
    *   [1.1 Formalizing the Economy](#1.1-Formalizing-the-Economy)
    *   [1.2 Existence Proofs and Fixed-Point Theorems](#1.2-Existence-Proofs-and-Fixed-Point-Theorems)
2.  [The Fundamental Theorems of Welfare Economics](#2.-The-Fundamental-Theorems-of-Welfare-Economics)
3.  [Computational General Equilibrium (CGE) Modeling](#3.-Computational-General-Equilibrium-(CGE)-Modeling)
    *   [3.1 A Simple CGE Model: Structure and Calibration](#3.1-A-Simple-CGE-Model:-Structure-and-Calibration)
    *   [3.2 The Negishi Method: An Alternative Solution Algorithm](#3.2-The-Negishi-Method:-An-Alternative-Solution-Algorithm)
4.  [Application: A Heckscher-Ohlin Model of International Trade](#4.-Application:-A-Heckscher-Ohlin-Model-of-International-Trade)
5.  [Chapter Summary](#5.-Chapter-Summary)
6.  [Exercises](#6.-Exercises)

### Introduction: Formalizing the Invisible Hand

The theory of **General Equilibrium (GE)**, pioneered by Léon Walras, provides the formal framework for analyzing the interdependencies of all markets in an economy. It addresses one of the oldest and most profound questions in economics: How can a decentralized market system, composed of millions of self-interested, independent agents, produce a coherent—and hopefully efficient—allocation of society's resources? 

This chapter explores the theoretical foundations of GE, its welfare implications, and its modern application in large-scale computational models.

We will cover:
1.  **The Arrow-Debreu Model:** A formal, abstract representation of a competitive economy.
2.  **Existence of Equilibrium:** The conditions required for a competitive equilibrium to exist, and the role of fixed-point theorems.
3.  **Welfare Theorems:** The formal link between competitive markets and Pareto efficiency.
4.  **Computational General Equilibrium (CGE):** The implementation and application of large-scale, multi-sector GE models for policy analysis, including alternative solution methods like the **Negishi algorithm**.

### 1. The Arrow-Debreu Model and Existence of Equilibrium

#### 1.1 Formalizing the Economy
The Arrow-Debreu model provides the most general formulation of a competitive economy. Its key innovation is its definition of a **commodity** as a good distinguished by its physical properties, its location, and the **date and state of nature** at which it becomes available. A single price vector therefore specifies prices for all goods, in all places, at all times, and in all possible future states of the world.

An **Arrow-Debreu equilibrium** consists of a price vector $\mathbf{p}^*$ for all commodities and an allocation for every agent such that:
1.  **Consumer Optimization:** Each consumer chooses their most preferred consumption bundle from their budget set.
2.  **Firm Optimization:** Each firm chooses its production plan to maximize profits.
3.  **Market Clearing:** The total demand for every commodity equals its total supply (from initial endowments and firm production).

#### 1.2 Existence Proofs and Fixed-Point Theorems
The proof of the existence of such an equilibrium, developed by Arrow, Debreu, and McKenzie, is a cornerstone of economic theory. It relies on **fixed-point theorems** applied to an **excess demand correspondence**, $Z(p)$.

The key insight is to construct a continuous function that maps a set of prices to a new set of prices, raising the price of goods in excess demand and lowering the price of those in excess supply. A fixed point of this map is a price vector where no prices need to change, which can only occur if all markets are in equilibrium ($Z(p^*) = 0$). To guarantee the existence of such a fixed point, the proof requires certain technical conditions, most importantly that consumer preferences are **convex** and continuous, and that production technologies are convex.

### 2. The Fundamental Theorems of Welfare Economics
These two theorems provide the formal link between competitive market outcomes and Pareto efficiency.

1.  **First Welfare Theorem:** Any competitive (Walrasian) equilibrium is Pareto efficient. This is the formal statement of Adam Smith's "invisible hand."
2.  **Second Welfare Theorem:** Any Pareto efficient allocation can be supported as a competitive equilibrium after a suitable redistribution of initial endowments. This theorem provides a powerful (though controversial) justification for separating the goals of efficiency and equity: policy can focus on redistribution of endowments, and then let the market achieve an efficient outcome.

### 3. Computational General Equilibrium (CGE) Modeling
Practical policy analysis requires numerical models. **Computational General Equilibrium (CGE)** models are large-scale numerical models that simulate the general equilibrium of an actual economy, often with dozens of sectors, multiple types of households, and a detailed specification of taxes and government policies.

#### 3.1 A Simple CGE Model: Structure and Calibration
A CGE model is defined by two key components:
1.  **Economic Data:** A **Social Accounting Matrix (SAM)**, which is a comprehensive, consistent snapshot of all economic flows in an economy for a base year.
2.  **Behavioral Assumptions:** A set of equations describing the behavior of agents, typically assuming CES utility and production functions. These functions are **calibrated** so that the model's equilibrium in the base year replicates the data in the SAM exactly.

We build a simple 2-sector (X, Y), 2-factor (K, L) model. The equilibrium is a set of prices $(P_X, P_Y, W, R)$ that solves a system of market-clearing equations.

In [ ]:
sec("A Simple CGE Model")

class CGEModel:
    def __init__(self, calib_data):
        self.p = calib_data # Parameters and base year data
    
    def equations(self, P, tax_rate=0.0):
        P_X, P_Y, R = P; W = 1.0 # Numeraire
        p = self.p
        P_X_producer = P_X / (1 + tax_rate)
        # Zero profit conditions
        eq1 = P_X_producer - (p['A_X'] * (W**p['alpha_X']) * (R**(1-p['alpha_X'])))
        eq2 = P_Y - (p['A_Y'] * (W**p['alpha_Y']) * (R**(1-p['alpha_Y'])))
        # Household income & demands
        I = W * p['L_bar'] + R * p['K_bar'] + tax_rate * P_X_producer * (p['beta_c'] * I / P_X)
        D_X = p['beta_c'] * I / P_X
        D_Y = (1 - p['beta_c']) * I / P_Y
        # Factor demands
        L_d = p['alpha_X'] * (P_X_producer / W) * D_X + p['alpha_Y'] * (P_Y / W) * D_Y
        K_d = (1-p['alpha_X']) * (P_X_producer / R) * D_X + (1-p['alpha_Y']) * (P_Y / R) * D_Y
        # Market clearing (Walras' Law means we only need N-1 equations)
        return [eq1, eq2, p['L_bar'] - L_d]

calibration = {'L_bar': 100, 'K_bar': 50, 'alpha_X': 0.6, 'alpha_Y': 0.4, 'beta_c': 0.5, 'A_X': 1.0, 'A_Y': 1.0}
model = CGEModel(calibration)

initial_guess = [1.0, 1.0, 1.0]
base_solution = root(model.equations, initial_guess)
note(f"Base Case Equilibrium (W=1): Px={base_solution.x[0]:.3f}, Py={base_solution.x[1]:.3f}, R={base_solution.x[2]:.3f}")

#### 3.2 The Negishi Method: An Alternative Solution Algorithm
Solving for market-clearing prices can be difficult. The **Negishi method** provides an alternative approach that can be more stable. It is based on the Second Welfare Theorem. Instead of searching for prices, it searches for the **Pareto weights** in a social planner's problem.

**Algorithm:**
1.  Define a social planner who maximizes a weighted sum of household utilities: $\max \sum_i \omega_i u_i(c_i)$, where $\omega_i$ are the Pareto weights.
2.  For a given set of weights $\boldsymbol{\omega}$, solve the planner's problem. This gives an allocation that is, by construction, Pareto efficient.
3.  From this allocation, calculate the implied market prices (the marginal rates of substitution).
4.  Check if each household can afford their allocated bundle at these prices, given their initial endowment. If not, adjust the Pareto weights (e.g., give more weight to households with a budget deficit) and repeat until the budget constraints are satisfied.

This method replaces the difficult price-finding problem with a search over the (often smaller) space of Pareto weights.

### 4. Application: A Heckscher-Ohlin Model of International Trade
We can use a two-country, two-good, two-factor CGE model to demonstrate the core theorems of international trade. We assume two countries, "Home" and "Foreign," have identical technologies and preferences but differ in their factor endowments. Home is relatively labor-abundant, and Foreign is relatively capital-abundant.

In [ ]:
sec("A 2x2x2 Heckscher-Ohlin Trade Model")
# Assume Good X is labor-intensive (high alpha), Good Y is capital-intensive (low alpha)
# Assume Home is labor-abundant, Foreign is capital-abundant
calib_home = {'L_bar': 120, 'K_bar': 40, 'alpha_X': 0.7, 'alpha_Y': 0.3, 'beta_c': 0.5, 'A_X': 1.0, 'A_Y': 1.0}
calib_fgn = {'L_bar': 50, 'K_bar': 50, 'alpha_X': 0.7, 'alpha_Y': 0.3, 'beta_c': 0.5, 'A_X': 1.0, 'A_Y': 1.0}
home_model = CGEModel(calib_home)
fgn_model = CGEModel(calib_fgn)

# 1. Solve for Autarky (no trade) equilibrium in each country
p_aut_home = root(home_model.equations, [1.0, 1.0, 1.0]).x
p_aut_fgn = root(fgn_model.equations, [1.0, 1.0, 1.0]).x
note(f"Autarky relative price of Good X (Px/Py) in Home: {p_aut_home[0]/p_aut_home[1]:.3f}")
note(f"Autarky relative price of Good X (Px/Py) in Foreign: {p_aut_fgn[0]/p_aut_fgn[1]:.3f}")
note("As predicted, the labor-abundant country (Home) has a lower relative price for the labor-intensive good (X).")

# 2. Solve for Free Trade equilibrium
def world_excess_demand(P):
    # Given world prices P, find each country's excess demand and sum them
    # ... (omitted for brevity, but involves solving each country's GE problem) ...
    # This is a complex fixed-point problem.
    pass
note("\nSolving for the full free-trade equilibrium requires a more complex algorithm to find world prices that clear markets globally. The result would show Home exporting Good X and Foreign exporting Good Y.")

### 5. Chapter Summary
- **General Equilibrium:** GE theory provides a formal framework for analyzing an entire economy where all markets are interdependent.
- **Existence and Welfare:** Fixed-point theorems are used to prove the existence of a competitive equilibrium. The two Fundamental Welfare Theorems establish the deep link between competitive markets and Pareto efficiency.
- **CGE Models:** These are large-scale numerical models used for quantitative policy analysis. They are built on a Social Accounting Matrix (SAM) and a set of calibrated behavioral equations.
- **Solution Methods:** The standard approach is to solve for a vector of market-clearing prices. The **Negishi method** provides an alternative by searching over the space of social planner weights.
- **Applications:** GE models are the workhorse for analyzing the economy-wide effects of policies like taxes, subsidies, and international trade liberalization.

### 6. Exercises

1.  **Stolper-Samuelson Theorem:** The Stolper-Samuelson theorem states that opening to trade (or imposing a tariff) will increase the real return to a country's relatively abundant factor and decrease the real return to its relatively scarce factor. In our Heckscher-Ohlin model, which factor is abundant in the Home country? When moving from autarky to free trade, does the real return to this factor (e.g., W/P_X) rise or fall?

2.  **Trade Policy:** Analyze the effect of a 20% tariff imposed by the Home country on imports of Good Y. How does this affect world prices, domestic prices, and the volume of trade?

3.  **Factor Intensity Reversal:** A "factor intensity reversal" occurs if a good is, say, labor-intensive at one set of factor prices but capital-intensive at another. Why might this complicate the predictions of the Heckscher-Ohlin model?

4.  **Negishi Method:** Explain in your own words the main advantage of the Negishi method over the standard Walrasian price-search algorithm. In what kind of models might this advantage be most significant?

5.  **CGE Model Calibration:** CGE models are typically calibrated to match a single year's data perfectly. Discuss the strengths and weaknesses of this calibration approach compared to a more statistical estimation method like GMM or MLE, which uses time-series data.